In [1]:
!pip install xlrd

In [2]:
import doctest
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.datasets import make_blobs
from scipy.stats import multivariate_normal
from matplotlib.patches import Ellipse
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
import re

# two useful data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
# setup plotting in a notebook in a reasonable way
%matplotlib inline
%config InlineBackend.figure_format='retina'

# default figure aesthetics I'll be using, 
# there are other choices, see seaborn docs
sns.set_style("white")
sns.set_context("notebook")

In [3]:
def remove_nums_from_string(input_string):
    return re.sub(r'\d', '', input_string)

In [4]:
us_state_dictionary = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISIANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY",
    "DISTRICT OF COLUMBIA": "DC",
}

### read in crime dataset 1

In [5]:
crime_40_60_df = pd.read_csv(r"archive (3)\crime_40_60.csv")
crime_40_60_df

,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft
0,Pennsylvania,"Abington Township, Montgomery County","55,731",197.4,1.8,14.4,70.0,111.2,1979.1,296.1,1650.8,32.3
1,Oregon,Albany,"51,084",86.1,0.0,19.6,45.0,21.5,3092.9,438.5,2470.4,184.0
2,Louisiana,Alexandria,"48,449",1682.2,18.6,28.9,293.1,1341.6,7492.4,2010.4,5102.3,379.8
3,California,Aliso Viejo,"48,999",87.8,0.0,0.0,12.2,75.5,847.0,208.2,612.3,26.5
4,Florida,Altamonte Springs,"42,296",335.7,2.4,21.3,82.8,229.3,3057.0,427.9,2463.6,165.5
...,...,...,...,...,...,...,...,...,...,...,...,...
353,California,Woodland,"56,323",314.3,0.0,17.8,83.4,213.1,3080.4,640.9,2080.9,358.6
354,Rhode Island,Woonsocket,"41,147",534.7,0.0,55.9,143.4,335.4,3059.8,840.9,2041.5,177.4
355,Texas,Wylie,"43,645",64.2,4.6,2.3,6.9,50.4,1404.5,229.1,1111.2,64.2
356,Pennsylvania,York,"43,955",1335.5,25.0,100.1,430.0,780.3,3599.1,985.1,2388.8,225.2


In [6]:
crime_60_100_df = pd.read_csv(r"archive (3)\crime_60_100.csv")
crime_60_100_df

,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft
0,California,Alameda,"75,467",212.0,1.3,11.91,106.0,92.8,"2,507.1",392.2,"1,723.9",390.9
1,Georgia,Albany,"78,512","1,035.5",5.1,34.4,285.3,710.7,"6,369.7","1,793.4","4,291.1",285.3
2,New York,Albany,"98,187",816.8,4.1,43.8,253.6,515.3,"4,420.1",903.4,"3,359.9",156.8
3,California,Alhambra,"84,469",176.4,-,2.4,78.1,95.9,"2,271.8",384.8,"1,585.2",301.9
4,Texas,Allen,"88,783",61.9,-,12.4,14.6,34.9,"1,612.9",242.2,"1,321.2",49.6
...,...,...,...,...,...,...,...,...,...,...,...,...
300,California,Yorba Linda,"65,820",80.5,-,4.6,16.7,59.3,"1,195.7",338.8,808.3,48.6
301,Pennsylvania,York Area Regional,"60,732",120.2,1.6,23.1,26.3,69.2,"1,052.2",260.2,759.1,32.9
302,Ohio,Youngstown,"66,567","1,027.5",33.0,58.6,366.5,569.4,"5,462.2","2,612.4","2,435.1",414.6
303,California,Yuba City,"65,653",376.2,6.1,33.5,65.5,271.1,"3,018.9",763.1,"1,975.5",280.3


In [7]:
for i in ['violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']:
    crime_60_100_df[i] = crime_60_100_df[i].str.replace(',', '')
    crime_60_100_df[i] = crime_60_100_df[i].astype(float)
crime_60_100_df

,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft
0,California,Alameda,"75,467",212.0,1.3,11.91,106.0,92.8,2507.1,392.2,1723.9,390.9
1,Georgia,Albany,"78,512",1035.5,5.1,34.4,285.3,710.7,6369.7,"1,793.4",4291.1,285.3
2,New York,Albany,"98,187",816.8,4.1,43.8,253.6,515.3,4420.1,903.4,3359.9,156.8
3,California,Alhambra,"84,469",176.4,-,2.4,78.1,95.9,2271.8,384.8,1585.2,301.9
4,Texas,Allen,"88,783",61.9,-,12.4,14.6,34.9,1612.9,242.2,1321.2,49.6
...,...,...,...,...,...,...,...,...,...,...,...,...
300,California,Yorba Linda,"65,820",80.5,-,4.6,16.7,59.3,1195.7,338.8,808.3,48.6
301,Pennsylvania,York Area Regional,"60,732",120.2,1.6,23.1,26.3,69.2,1052.2,260.2,759.1,32.9
302,Ohio,Youngstown,"66,567",1027.5,33.0,58.6,366.5,569.4,5462.2,"2,612.4",2435.1,414.6
303,California,Yuba City,"65,653",376.2,6.1,33.5,65.5,271.1,3018.9,763.1,1975.5,280.3


In [8]:
crime_100_250_df = pd.read_csv(r"archive (3)\crime_100_250.csv")
crime_100_250_df

,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft
0,Texas,Abilene,"119,886",393.7,2.5,31.7,105.9,253.6,"3,664.3",865,"2,656.7",142.6
1,Ohio,Akron,"198,390",886.6,12.1,84.2,290.8,499.5,"5,057.7","1,728.4","2,965.9",363.4
2,Virginia,Alexandria,"145,892",166.6,-,6.2,94.6,65.8,"2,049.5",192.6,"1,633.4",223.5
3,Pennsylvania,Allentown,"119,334",547.2,12.6,45.3,313.4,176,"3,857.2","1,045.8","2,503.1",308.4
4,Texas,Amarillo,"196,576",650.1,5.1,56.0,141.4,447.7,"4,527.5","1,061.7","3,145.9",320
...,...,...,...,...,...,...,...,...,...,...,...,...
207,North Carolina,Wilmington,"109,370",565.1,7.3,21.0,238.6,298.1,"5,403.7","1,548.9","3,513.8",341
208,North Carolina,Winston-Salem,"234,687",663,2.6,31.5,182.4,446.6,"5,838","1,929.8","3,617.6",290.6
209,New Jersey,Woodbridge Township,"100,612",121.3,-,3.0,54.7,63.6,"1,955",335.9,"1,504.8",114.3
210,Massachusetts,Worcester,"183,247",959.4,4.4,18.0,228.7,708.3,"3,510","1,113.3","2,143",253.8


In [9]:
for i in ['violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']:
    crime_100_250_df[i] = crime_100_250_df[i].str.replace(',', '')
    crime_100_250_df[i] = crime_100_250_df[i].astype(float)
crime_100_250_df

,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft
0,Texas,Abilene,"119,886",393.7,2.5,31.7,105.9,253.6,3664.3,865,2656.7,142.6
1,Ohio,Akron,"198,390",886.6,12.1,84.2,290.8,499.5,5057.7,"1,728.4",2965.9,363.4
2,Virginia,Alexandria,"145,892",166.6,-,6.2,94.6,65.8,2049.5,192.6,1633.4,223.5
3,Pennsylvania,Allentown,"119,334",547.2,12.6,45.3,313.4,176,3857.2,"1,045.8",2503.1,308.4
4,Texas,Amarillo,"196,576",650.1,5.1,56.0,141.4,447.7,4527.5,"1,061.7",3145.9,320.0
...,...,...,...,...,...,...,...,...,...,...,...,...
207,North Carolina,Wilmington,"109,370",565.1,7.3,21.0,238.6,298.1,5403.7,"1,548.9",3513.8,341.0
208,North Carolina,Winston-Salem,"234,687",663.0,2.6,31.5,182.4,446.6,5838.0,"1,929.8",3617.6,290.6
209,New Jersey,Woodbridge Township,"100,612",121.3,-,3.0,54.7,63.6,1955.0,335.9,1504.8,114.3
210,Massachusetts,Worcester,"183,247",959.4,4.4,18.0,228.7,708.3,3510.0,"1,113.3",2143.0,253.8


In [10]:
crime_250_plus_df = pd.read_csv(r"archive (3)\crime_250_plus.csv")
crime_250_plus_df.rename(columns = {'tot_violent_crime':'violent_crime'}, inplace=True)
crime_250_plus_df.rename(columns = {'tot_prop_crim':'prop_crime'}, inplace=True)
crime_250_plus_df

,states,cities,population,total_crime,murder,rape,robbery,agrv_assault,violent_crime,burglary,larceny,vehicle_theft,prop_crime,arson
0,Alabama,Mobile3,"248,431",6217.02,20.13,58.16,177.11,485.85,740.25,"1,216.84","3,730.21",506.78,"5,453.83",22.94
1,Alaska,Anchorage,"296,188",6640.04,9.12,132.01,262.67,799.49,"1,203.29",748.17,"3,619.66","1,047.98","5,415.82",20.93
2,Arizona,Chandler,"249,355",2589.08,2.01,52.13,56.95,148.38,259.47,314.41,"1,866.01",149.18,"2,329.61",NaN
3,Arizona,Gilbert,"242,090",1483.75,2.07,16.11,21.07,46.26,85.51,192.49,"1,137.59",55.76,"1,385.85",12.39
4,Arizona,Glendale,"249,273",5037.85,4.81,38.91,192.96,251.53,488.22,637.45,"3,426.36",466.56,"4,530.37",19.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Virginia,Virginia Beach,"454,353",2083.40,3.08,22.45,59.87,52.16,137.56,165.95,"1,667.65",101.02,"1,934.62",11.22
96,Washington,Seattle,"721,365",5911.99,3.74,36.87,210.02,382.05,632.69,"1,081.98","3,673.45",503.21,"5,258.64",20.66
97,Washington,Spokane,"217,066",7881.01,2.76,109.64,105.50,408.63,626.54,"1,023.19","5,408.03",800.22,"7,231.44",23.03
98,Wisconsin,Madison,"255,850",3040.46,4.30,37.13,82.86,250.15,374.44,362.32,"2,130.94",168.85,"2,662.11",3.91


In [11]:
for i in ['violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']:
    crime_250_plus_df[i] = crime_250_plus_df[i].str.replace(',', '')
    crime_250_plus_df[i] = crime_250_plus_df[i].astype(float)
crime_250_plus_df

,states,cities,population,total_crime,murder,rape,robbery,agrv_assault,violent_crime,burglary,larceny,vehicle_theft,prop_crime,arson
0,Alabama,Mobile3,"248,431",6217.02,20.13,58.16,177.11,485.85,740.25,"1,216.84",3730.21,506.78,5453.83,22.94
1,Alaska,Anchorage,"296,188",6640.04,9.12,132.01,262.67,799.49,1203.29,748.17,3619.66,1047.98,5415.82,20.93
2,Arizona,Chandler,"249,355",2589.08,2.01,52.13,56.95,148.38,259.47,314.41,1866.01,149.18,2329.61,NaN
3,Arizona,Gilbert,"242,090",1483.75,2.07,16.11,21.07,46.26,85.51,192.49,1137.59,55.76,1385.85,12.39
4,Arizona,Glendale,"249,273",5037.85,4.81,38.91,192.96,251.53,488.22,637.45,3426.36,466.56,4530.37,19.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Virginia,Virginia Beach,"454,353",2083.40,3.08,22.45,59.87,52.16,137.56,165.95,1667.65,101.02,1934.62,11.22
96,Washington,Seattle,"721,365",5911.99,3.74,36.87,210.02,382.05,632.69,"1,081.98",3673.45,503.21,5258.64,20.66
97,Washington,Spokane,"217,066",7881.01,2.76,109.64,105.50,408.63,626.54,"1,023.19",5408.03,800.22,7231.44,23.03
98,Wisconsin,Madison,"255,850",3040.46,4.30,37.13,82.86,250.15,374.44,362.32,2130.94,168.85,2662.11,3.91


### Concat all 4 datasets into 1

In [12]:
crime_combined_df = pd.DataFrame()

In [13]:
crime_df_list = ['crime_40_60_df', 'crime_60_100_df', 'crime_100_250_df', 'crime_250_plus_df']

In [14]:
crime_combined_df = pd.concat([crime_40_60_df, crime_60_100_df, crime_100_250_df, crime_250_plus_df])
crime_combined_df = crime_combined_df.reset_index()
crime_combined_df

,index,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft,total_crime,arson
0,0,Pennsylvania,"Abington Township, Montgomery County","55,731",197.40,1.8,14.4,70.00,111.2,1979.10,296.1,1650.80,32.30,NaN,NaN
1,1,Oregon,Albany,"51,084",86.10,0.0,19.6,45.00,21.5,3092.90,438.5,2470.40,184.00,NaN,NaN
2,2,Louisiana,Alexandria,"48,449",1682.20,18.6,28.9,293.10,1341.6,7492.40,2010.4,5102.30,379.80,NaN,NaN
3,3,California,Aliso Viejo,"48,999",87.80,0.0,0.0,12.20,75.5,847.00,208.2,612.30,26.50,NaN,NaN
4,4,Florida,Altamonte Springs,"42,296",335.70,2.4,21.3,82.80,229.3,3057.00,427.9,2463.60,165.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,95,Virginia,Virginia Beach,"454,353",137.56,3.08,22.45,59.87,52.16,1934.62,165.95,1667.65,101.02,2083.40,11.22
971,96,Washington,Seattle,"721,365",632.69,3.74,36.87,210.02,382.05,5258.64,"1,081.98",3673.45,503.21,5911.99,20.66
972,97,Washington,Spokane,"217,066",626.54,2.76,109.64,105.50,408.63,7231.44,"1,023.19",5408.03,800.22,7881.01,23.03
973,98,Wisconsin,Madison,"255,850",374.44,4.3,37.13,82.86,250.15,2662.11,362.32,2130.94,168.85,3040.46,3.91


In [15]:
state_list_revised_1 = []
cities_list_revised_1 = []
for i,j in zip(crime_combined_df['states'], crime_combined_df['cities']):
    state_list_revised_1.append(remove_nums_from_string(i))
    cities_list_revised_1.append(remove_nums_from_string(j))
crime_combined_df['states'] = state_list_revised_1
crime_combined_df['cities'] = cities_list_revised_1
crime_combined_df

,index,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft,total_crime,arson
0,0,Pennsylvania,"Abington Township, Montgomery County","55,731",197.40,1.8,14.4,70.00,111.2,1979.10,296.1,1650.80,32.30,NaN,NaN
1,1,Oregon,Albany,"51,084",86.10,0.0,19.6,45.00,21.5,3092.90,438.5,2470.40,184.00,NaN,NaN
2,2,Louisiana,Alexandria,"48,449",1682.20,18.6,28.9,293.10,1341.6,7492.40,2010.4,5102.30,379.80,NaN,NaN
3,3,California,Aliso Viejo,"48,999",87.80,0.0,0.0,12.20,75.5,847.00,208.2,612.30,26.50,NaN,NaN
4,4,Florida,Altamonte Springs,"42,296",335.70,2.4,21.3,82.80,229.3,3057.00,427.9,2463.60,165.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,95,Virginia,Virginia Beach,"454,353",137.56,3.08,22.45,59.87,52.16,1934.62,165.95,1667.65,101.02,2083.40,11.22
971,96,Washington,Seattle,"721,365",632.69,3.74,36.87,210.02,382.05,5258.64,"1,081.98",3673.45,503.21,5911.99,20.66
972,97,Washington,Spokane,"217,066",626.54,2.76,109.64,105.50,408.63,7231.44,"1,023.19",5408.03,800.22,7881.01,23.03
973,98,Wisconsin,Madison,"255,850",374.44,4.3,37.13,82.86,250.15,2662.11,362.32,2130.94,168.85,3040.46,3.91


In [16]:
crime_combined_df['states'].unique()

array(['Pennsylvania', 'Oregon', 'Louisiana', 'California', 'Florida',
       'Iowa', 'Indiana', 'Minnesota', 'Massachusetts', 'Alabama',
       'Illinois', 'Tennessee', 'Ohio', 'Texas', 'Nebraska',
       'Mississippi', 'New York', 'Virginia', 'Michigan', 'New Jersey',
       'Missouri', 'Utah', 'Maryland', 'Kentucky', 'Colorado', 'Arizona',
       'North Carolina', 'Idaho', 'Wyoming', 'New Hampshire', 'Georgia',
       'Connecticut', 'Rhode Island', 'Washington', 'Oklahoma',
       'New Mexico', 'Wisconsin', 'North Dakota', 'Montana', 'Kansas',
       'Arkansas', 'South Carolina', 'Maine', 'South Dakota', 'Nevada',
       'Delaware', '\xa0Alabama', '\xa0Alaska', '\xa0Arizona',
       '\xa0California', '\xa0Colorado', '\xa0District of Columbia',
       '\xa0Florida', '\xa0Georgia', '\xa0Hawaii', '\xa0Idaho',
       '\xa0Illinois', '\xa0Indiana', '\xa0Iowa', '\xa0Kansas',
       '\xa0Kentucky', '\xa0Louisiana', '\xa0Maryland',
       '\xa0Massachusetts', '\xa0Michigan', '\xa0Minnesota'

### remove '\xa0' from string; leave only state name

In [17]:
state_df = []
for i in crime_combined_df['states']:
    if '\xa0' in i:
        state_df.append(i.replace('\xa0', ''))
    else:
        state_df.append(i)
state_df = [state.upper() for state in state_df]
state_df

['PENNSYLVANIA',
 'OREGON',
 'LOUISIANA',
 'CALIFORNIA',
 'FLORIDA',
 'PENNSYLVANIA',
 'IOWA',
 'INDIANA',
 'IOWA',
 'FLORIDA',
 'MINNESOTA',
 'CALIFORNIA',
 'MASSACHUSETTS',
 'MASSACHUSETTS',
 'ALABAMA',
 'CALIFORNIA',
 'MASSACHUSETTS',
 'ILLINOIS',
 'TENNESSEE',
 'OHIO',
 'TEXAS',
 'CALIFORNIA',
 'NEBRASKA',
 'ILLINOIS',
 'MASSACHUSETTS',
 'MASSACHUSETTS',
 'MISSISSIPPI',
 'NEW YORK',
 'VIRGINIA',
 'MINNESOTA',
 'MICHIGAN',
 'NEW JERSEY',
 'MISSOURI',
 'OHIO',
 'UTAH',
 'MARYLAND',
 'KENTUCKY',
 'FLORIDA',
 'CALIFORNIA',
 'CALIFORNIA',
 'NEW JERSEY',
 'PENNSYLVANIA',
 'MASSACHUSETTS',
 'COLORADO',
 'ARIZONA',
 'ILLINOIS',
 'ARIZONA',
 'NORTH CAROLINA',
 'IDAHO',
 'CALIFORNIA',
 'ILLINOIS',
 'ARIZONA',
 'WYOMING',
 'COLORADO',
 'CALIFORNIA',
 'TEXAS',
 'TEXAS',
 'CALIFORNIA',
 'CALIFORNIA',
 'NORTH CAROLINA',
 'VIRGINIA',
 'MICHIGAN',
 'MISSOURI',
 'MASSACHUSETTS',
 'OHIO',
 'TENNESSEE',
 'CALIFORNIA',
 'FLORIDA',
 'IDAHO',
 'OHIO',
 'TENNESSEE',
 'CALIFORNIA',
 'INDIANA',
 'COLORADO'

### Change state name to its corresponding abbreviation

In [18]:
state_abbrev = [us_state_dictionary[key] for key in state_df]
state_abbrev

['PA',
 'OR',
 'LA',
 'CA',
 'FL',
 'PA',
 'IA',
 'IN',
 'IA',
 'FL',
 'MN',
 'CA',
 'MA',
 'MA',
 'AL',
 'CA',
 'MA',
 'IL',
 'TN',
 'OH',
 'TX',
 'CA',
 'NE',
 'IL',
 'MA',
 'MA',
 'MS',
 'NY',
 'VA',
 'MN',
 'MI',
 'NJ',
 'MO',
 'OH',
 'UT',
 'MD',
 'KY',
 'FL',
 'CA',
 'CA',
 'NJ',
 'PA',
 'MA',
 'CO',
 'AZ',
 'IL',
 'AZ',
 'NC',
 'ID',
 'CA',
 'IL',
 'AZ',
 'WY',
 'CO',
 'CA',
 'TX',
 'TX',
 'CA',
 'CA',
 'NC',
 'VA',
 'MI',
 'MO',
 'MA',
 'OH',
 'TN',
 'CA',
 'FL',
 'ID',
 'OH',
 'TN',
 'CA',
 'IN',
 'CO',
 'NH',
 'TX',
 'TX',
 'FL',
 'OR',
 'CA',
 'IL',
 'CA',
 'FL',
 'OH',
 'CA',
 'CA',
 'VA',
 'IL',
 'MI',
 'AL',
 'CA',
 'IL',
 'TX',
 'CA',
 'FL',
 'IL',
 'UT',
 'CA',
 'OH',
 'IA',
 'GA',
 'NJ',
 'CT',
 'MI',
 'RI',
 'CA',
 'MN',
 'WA',
 'NJ',
 'CA',
 'IL',
 'CT',
 'OK',
 'TX',
 'MA',
 'NJ',
 'OH',
 'NM',
 'MA',
 'MO',
 'WI',
 'FL',
 'CA',
 'NY',
 'TX',
 'CA',
 'TX',
 'CA',
 'CA',
 'IL',
 'ND',
 'NE',
 'CO',
 'TX',
 'MT',
 'NY',
 'IN',
 'NJ',
 'MD',
 'TX',
 'NY',
 'CA',
 'PA',

In [19]:
crime_combined_df['states'] = state_abbrev
crime_combined_df

,index,states,cities,population,violent_crime,murder,rape,robbery,agrv_assault,prop_crime,burglary,larceny,vehicle_theft,total_crime,arson
0,0,PA,"Abington Township, Montgomery County","55,731",197.40,1.8,14.4,70.00,111.2,1979.10,296.1,1650.80,32.30,NaN,NaN
1,1,OR,Albany,"51,084",86.10,0.0,19.6,45.00,21.5,3092.90,438.5,2470.40,184.00,NaN,NaN
2,2,LA,Alexandria,"48,449",1682.20,18.6,28.9,293.10,1341.6,7492.40,2010.4,5102.30,379.80,NaN,NaN
3,3,CA,Aliso Viejo,"48,999",87.80,0.0,0.0,12.20,75.5,847.00,208.2,612.30,26.50,NaN,NaN
4,4,FL,Altamonte Springs,"42,296",335.70,2.4,21.3,82.80,229.3,3057.00,427.9,2463.60,165.50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,95,VA,Virginia Beach,"454,353",137.56,3.08,22.45,59.87,52.16,1934.62,165.95,1667.65,101.02,2083.40,11.22
971,96,WA,Seattle,"721,365",632.69,3.74,36.87,210.02,382.05,5258.64,"1,081.98",3673.45,503.21,5911.99,20.66
972,97,WA,Spokane,"217,066",626.54,2.76,109.64,105.50,408.63,7231.44,"1,023.19",5408.03,800.22,7881.01,23.03
973,98,WI,Madison,"255,850",374.44,4.3,37.13,82.86,250.15,2662.11,362.32,2130.94,168.85,3040.46,3.91


### Upload housing dataset

In [20]:
cleaned_housing_data = pd.read_csv('cleaned_housing_data.csv')
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans,"(30.0687, -89.9288)",121.001418
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans,"(30.0687, -89.9288)",121.980045
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham,"(33.5279, -86.7971)",15.987941
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham,"(33.5279, -86.7971)",16.708137
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)",87.664428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden,"(41.2279, -111.9682)",132.340884
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden,"(41.2279, -111.9682)",147.269952
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden,"(41.2279, -111.9682)",241.450936
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver,"(39.762, -104.8758)",227.287741


### Read in crime dataset 2

In [21]:
# note values in fbi_crime_df are total number of certain crime reported in a city
fbi_crime_df = pd.read_excel(r"Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_2019.xls")
fbi_crime_df = fbi_crime_df.drop([0,1])
fbi_crime_df = fbi_crime_df.reset_index(drop = True)
fbi_crime_df.columns = fbi_crime_df.iloc[0]
fbi_crime_df = fbi_crime_df[1:-8]
fbi_crime_df

,State,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
1,ALABAMA3,Hoover,85670,114,4,15,27,68,1922,128,1694,100,2
2,ALASKA,Anchorage,287731,3581,32,540,621,2388,12261,1692,9038,1531,93
3,NaN,Bethel,6544,130,1,47,3,79,132,20,84,28,12
4,NaN,Bristol Bay Borough,852,2,0,0,0,2,20,5,8,7,0
5,NaN,Cordova,2150,0,0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101,NaN,Sheridan,17895,9,0,4,0,5,369,75,278,16,3
8102,NaN,Thermopolis5,2830,13,0,0,0,13,34,7,22,5,0
8103,NaN,Torrington,6709,13,0,4,1,8,48,8,40,0,0
8104,NaN,Wheatland,3544,7,0,1,0,6,72,24,45,3,0


In [22]:
fbi_crime_df.columns

Index(['State', 'City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter', 'Rape1', 'Robbery',
       'Aggravated\nassault', 'Property\ncrime', 'Burglary', 'Larceny-\ntheft',
       'Motor\nvehicle\ntheft', 'Arson2'],
      dtype='object', name=0)

In [23]:
fbi_crime_df['State']

1       ALABAMA3
2         ALASKA
3            NaN
4            NaN
5            NaN
          ...   
8101         NaN
8102         NaN
8103         NaN
8104         NaN
8105         NaN
Name: State, Length: 8105, dtype: object

### Fill in state nan values with its actual corresponding state

In [24]:
state_list = []
current_state = 'ALABAMA'
for i in fbi_crime_df['State']:
    if pd.notna(i):
        current_state = i
    else:
        i = current_state
    state_list.append(i)
state_list        

['ALABAMA3',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZ

In [25]:
state_list_revised = []
for i in state_list:
    state_list_revised.append(remove_nums_from_string(i))
state_list_revised

['ALABAMA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ALASKA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZONA',
 'ARIZO

### Replace state name with its corresponding abbreviation

In [26]:
state_list_revised_abreviation = [us_state_dictionary[key] for key in state_list_revised]
len(state_list_revised_abreviation)

8105

In [27]:
fbi_crime_df['State'] = state_list_revised_abreviation
fbi_crime_df

,State,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
1,AL,Hoover,85670,114,4,15,27,68,1922,128,1694,100,2
2,AK,Anchorage,287731,3581,32,540,621,2388,12261,1692,9038,1531,93
3,AK,Bethel,6544,130,1,47,3,79,132,20,84,28,12
4,AK,Bristol Bay Borough,852,2,0,0,0,2,20,5,8,7,0
5,AK,Cordova,2150,0,0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101,WY,Sheridan,17895,9,0,4,0,5,369,75,278,16,3
8102,WY,Thermopolis5,2830,13,0,0,0,13,34,7,22,5,0
8103,WY,Torrington,6709,13,0,4,1,8,48,8,40,0,0
8104,WY,Wheatland,3544,7,0,1,0,6,72,24,45,3,0


In [28]:
crime_city_list = []
for i in fbi_crime_df['City']:
    crime_city_list.append(remove_nums_from_string(i))
crime_city_list

['Hoover',
 'Anchorage',
 'Bethel',
 'Bristol Bay Borough',
 'Cordova',
 'Craig',
 'Dillingham',
 'Fairbanks',
 'Haines',
 'Homer',
 'Juneau',
 'Kenai',
 'Ketchikan',
 'Kodiak',
 'Kotzebue',
 'Nome',
 'North Pole',
 'North Slope Borough',
 'Palmer',
 'Petersburg',
 'Seward',
 'Sitka',
 'Skagway',
 'Soldotna',
 'Unalaska',
 'Valdez',
 'Wasilla',
 'Wrangell',
 'Apache Junction',
 'Avondale',
 'Buckeye',
 'Bullhead City',
 'Camp Verde',
 'Casa Grande',
 'Chandler',
 'Chino Valley',
 'Clarkdale',
 'Coolidge',
 'Cottonwood',
 'Douglas',
 'Eagar',
 'El Mirage',
 'Flagstaff',
 'Florence',
 'Fredonia',
 'Gilbert',
 'Glendale',
 'Globe',
 'Goodyear',
 'Hayden',
 'Holbrook',
 'Huachuca City',
 'Jerome',
 'Kearny',
 'Kingman',
 'Lake Havasu City',
 'Mammoth',
 'Marana',
 'Maricopa',
 'Mesa',
 'Miami',
 'Nogales',
 'Oro Valley',
 'Page',
 'Paradise Valley',
 'Parker',
 'Payson',
 'Peoria',
 'Phoenix',
 'Pima',
 'Pinetop-Lakeside',
 'Prescott',
 'Prescott Valley',
 'Safford',
 'Sahuarita',
 'San Lu

In [29]:
fbi_crime_df['City'] = crime_city_list
fbi_crime_df

,State,City,Population,Violent\ncrime,Murder and\nnonnegligent\nmanslaughter,Rape1,Robbery,Aggravated\nassault,Property\ncrime,Burglary,Larceny-\ntheft,Motor\nvehicle\ntheft,Arson2
1,AL,Hoover,85670,114,4,15,27,68,1922,128,1694,100,2
2,AK,Anchorage,287731,3581,32,540,621,2388,12261,1692,9038,1531,93
3,AK,Bethel,6544,130,1,47,3,79,132,20,84,28,12
4,AK,Bristol Bay Borough,852,2,0,0,0,2,20,5,8,7,0
5,AK,Cordova,2150,0,0,0,0,0,7,1,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101,WY,Sheridan,17895,9,0,4,0,5,369,75,278,16,3
8102,WY,Thermopolis,2830,13,0,0,0,13,34,7,22,5,0
8103,WY,Torrington,6709,13,0,4,1,8,48,8,40,0,0
8104,WY,Wheatland,3544,7,0,1,0,6,72,24,45,3,0


### Subset crime dataset 1 to contain only certain columns

In [30]:
# just keep violent crime and property crime
crime_combined_df_revised = crime_combined_df[['states', 'cities', 'population', 'violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']]
crime_combined_df_revised = crime_combined_df_revised.dropna()
crime_combined_df_revised.rename(columns = {'states': 'State', 'cities': 'City'}, inplace=True)
crime_combined_df_revised

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,PA,"Abington Township, Montgomery County","55,731",197.40,1979.10,1650.80,32.30
1,OR,Albany,"51,084",86.10,3092.90,2470.40,184.00
2,LA,Alexandria,"48,449",1682.20,7492.40,5102.30,379.80
3,CA,Aliso Viejo,"48,999",87.80,847.00,612.30,26.50
4,FL,Altamonte Springs,"42,296",335.70,3057.00,2463.60,165.50
...,...,...,...,...,...,...,...
970,VA,Virginia Beach,"454,353",137.56,1934.62,1667.65,101.02
971,WA,Seattle,"721,365",632.69,5258.64,3673.45,503.21
972,WA,Spokane,"217,066",626.54,7231.44,5408.03,800.22
973,WI,Madison,"255,850",374.44,2662.11,2130.94,168.85


In [31]:
crime_combined_df_revised['population'] = crime_combined_df_revised['population'].str.replace(',', '')
crime_combined_df_revised['population'] = crime_combined_df_revised['population'].astype(float)
crime_combined_df_revised = crime_combined_df_revised.reset_index(drop = True)
crime_combined_df_revised

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,PA,"Abington Township, Montgomery County",55731.0,197.40,1979.10,1650.80,32.30
1,OR,Albany,51084.0,86.10,3092.90,2470.40,184.00
2,LA,Alexandria,48449.0,1682.20,7492.40,5102.30,379.80
3,CA,Aliso Viejo,48999.0,87.80,847.00,612.30,26.50
4,FL,Altamonte Springs,42296.0,335.70,3057.00,2463.60,165.50
...,...,...,...,...,...,...,...
945,VA,Virginia Beach,454353.0,137.56,1934.62,1667.65,101.02
946,WA,Seattle,721365.0,632.69,5258.64,3673.45,503.21
947,WA,Spokane,217066.0,626.54,7231.44,5408.03,800.22
948,WI,Madison,255850.0,374.44,2662.11,2130.94,168.85


#### Normalize all crimes by the total population in each city (i.e. per capita)

In [32]:
for i in range(crime_combined_df_revised.shape[0]):
    for j in ['violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']:
        crime_combined_df_revised[j][i] = crime_combined_df_revised[j].iloc[i] / crime_combined_df_revised['population'].iloc[i]
crime_combined_df_revised

C:\Users\krist\AppData\Local\Temp\ipykernel_20732\1355395394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_combined_df_revised[j][i] = crime_combined_df_revised[j].iloc[i] / crime_combined_df_revised['population'].iloc[i]


,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,PA,"Abington Township, Montgomery County",55731.0,0.003542,0.035512,0.029621,0.000580
1,OR,Albany,51084.0,0.001685,0.060545,0.048360,0.003602
2,LA,Alexandria,48449.0,0.034721,0.154645,0.105313,0.007839
3,CA,Aliso Viejo,48999.0,0.001792,0.017286,0.012496,0.000541
4,FL,Altamonte Springs,42296.0,0.007937,0.072276,0.058247,0.003913
...,...,...,...,...,...,...,...
945,VA,Virginia Beach,454353.0,0.000303,0.004258,0.003670,0.000222
946,WA,Seattle,721365.0,0.000877,0.007290,0.005092,0.000698
947,WA,Spokane,217066.0,0.002886,0.033314,0.024914,0.003687
948,WI,Madison,255850.0,0.001464,0.010405,0.008329,0.000660


In [33]:
fbi_crime_df_revised = fbi_crime_df[['State', 'City', 'Population', 'Violent\ncrime', 'Property\ncrime', 'Larceny-\ntheft', 'Motor\nvehicle\ntheft']]
fbi_crime_df_revised

,State,City,Population,Violent\ncrime,Property\ncrime,Larceny-\ntheft,Motor\nvehicle\ntheft
1,AL,Hoover,85670,114,1922,1694,100
2,AK,Anchorage,287731,3581,12261,9038,1531
3,AK,Bethel,6544,130,132,84,28
4,AK,Bristol Bay Borough,852,2,20,8,7
5,AK,Cordova,2150,0,7,6,0
...,...,...,...,...,...,...,...
8101,WY,Sheridan,17895,9,369,278,16
8102,WY,Thermopolis,2830,13,34,22,5
8103,WY,Torrington,6709,13,48,40,0
8104,WY,Wheatland,3544,7,72,45,3


In [34]:
fbi_rename_map = {
    'State':'State', 
    'City':'City', 
    'Population':'population', 
    'Violent\ncrime':'violent_crime', 
    'Property\ncrime':'prop_crime', 
    'Larceny-\ntheft':'larceny', 
    'Motor\nvehicle\ntheft':'vehicle_theft'
}

In [35]:
fbi_crime_df_revised.rename(columns = fbi_rename_map, inplace=True)
fbi_crime_df_revised = fbi_crime_df_revised.dropna()
fbi_crime_df_revised = fbi_crime_df_revised.reset_index(drop = True)
fbi_crime_df_revised

C:\Users\krist\AppData\Local\Temp\ipykernel_20732\2981303520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fbi_crime_df_revised.rename(columns = fbi_rename_map, inplace=True)


,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,AL,Hoover,85670,114,1922,1694,100
1,AK,Anchorage,287731,3581,12261,9038,1531
2,AK,Bethel,6544,130,132,84,28
3,AK,Bristol Bay Borough,852,2,20,8,7
4,AK,Cordova,2150,0,7,6,0
...,...,...,...,...,...,...,...
8054,WY,Sheridan,17895,9,369,278,16
8055,WY,Thermopolis,2830,13,34,22,5
8056,WY,Torrington,6709,13,48,40,0
8057,WY,Wheatland,3544,7,72,45,3


In [36]:
for i in range(fbi_crime_df_revised.shape[0]):
    for j in ['violent_crime', 'prop_crime', 'larceny', 'vehicle_theft']:
        fbi_crime_df_revised[j][i] = fbi_crime_df_revised[j].iloc[i] / fbi_crime_df_revised['population'].iloc[i]
fbi_crime_df_revised

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,AL,Hoover,85670,0.001331,0.022435,0.019774,0.001167
1,AK,Anchorage,287731,0.012446,0.042613,0.031411,0.005321
2,AK,Bethel,6544,0.019866,0.020171,0.012836,0.004279
3,AK,Bristol Bay Borough,852,0.002347,0.023474,0.00939,0.008216
4,AK,Cordova,2150,0.0,0.003256,0.002791,0.0
...,...,...,...,...,...,...,...
8054,WY,Sheridan,17895,0.000503,0.02062,0.015535,0.000894
8055,WY,Thermopolis,2830,0.004594,0.012014,0.007774,0.001767
8056,WY,Torrington,6709,0.001938,0.007155,0.005962,0.0
8057,WY,Wheatland,3544,0.001975,0.020316,0.012698,0.000847


In [37]:
cleaned_housing_data

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,military,incorporated,timezone,ranking,zips,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles
0,AL,Saraland,Scott Dr,36571.0,4.0,2.0,1614.0,148.636927,0.3805,240600.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.819534, -88.09596)",New Orleans,"(30.0687, -89.9288)",121.001418
1,AL,Saraland,Deer Run Dr,36571.0,3.0,2.0,1828.0,108.315098,0.4658,199700.0,...,False,True,America/Chicago,3,36571,1840015886,"(30.841074, -88.08948)",New Orleans,"(30.0687, -89.9288)",121.980045
2,AL,Chelsea,Mallet Way,35043.0,3.0,3.0,2224.0,150.629496,0.2600,336200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.357986, -86.6087)",Birmingham,"(33.5279, -86.7971)",15.987941
3,AL,Chelsea,Highway 440,35043.0,3.0,2.0,1425.0,195.789474,1.8200,280200.0,...,False,True,America/Chicago,3,35051 35043,1840014823,"(33.354427, -86.59509)",Birmingham,"(33.5279, -86.7971)",16.708137
4,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,False,True,America/Chicago,2,36112 36113 36110 36111 36116 36114 36115 3610...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)",87.664428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12599,WY,Big Piney,Taylor Ave,83113.0,4.0,2.0,1820.0,112.637363,0.1600,189800.0,...,False,True,America/Denver,3,83113,1840020061,"(42.552986, -110.113075)",Ogden,"(41.2279, -111.9682)",132.340884
12600,WY,Daniel,Booth Dr,83115.0,1.0,1.0,1008.0,337.202381,10.0600,307500.0,...,False,False,America/Denver,3,83115,1840018659,"(42.975636, -110.32426)",Ogden,"(41.2279, -111.9682)",147.269952
12601,WY,Shoshoni,Pine St,82649.0,6.0,3.0,3294.0,74.377656,0.3200,236800.0,...,False,True,America/Denver,3,82649,1840022355,"(43.23948, -108.11593)",Ogden,"(41.2279, -111.9682)",241.450936
12602,WY,Mills,N 4th Ave,82604.0,4.0,2.0,2120.0,90.094340,0.3100,184600.0,...,False,True,America/Denver,3,82644 82604 82601,1840022359,"(42.851006, -106.39181)",Denver,"(39.762, -104.8758)",227.287741


### Merge the crime datasets together

In [38]:
# if values in dataset 1 are in dataset 2 delete that value from dataset 1
crime_df_merged = pd.merge(crime_combined_df_revised, fbi_crime_df_revised, on = ['State', 'City'], how = 'outer', indicator = True)
crime_df_merged

,State,City,population_x,violent_crime_x,prop_crime_x,larceny_x,vehicle_theft_x,population_y,violent_crime_y,prop_crime_y,larceny_y,vehicle_theft_y,_merge
0,PA,"Abington Township, Montgomery County",55731.0,0.003542,0.035512,0.029621,0.000580,55476,0.000955,0.016836,0.015917,0.000342,both
1,OR,Albany,51084.0,0.001685,0.060545,0.048360,0.003602,54993,0.001273,0.026676,0.022585,0.001764,both
2,LA,Alexandria,48449.0,0.034721,0.154645,0.105313,0.007839,46630,0.015698,0.089642,0.065537,0.004975,both
3,CA,Aliso Viejo,48999.0,0.001792,0.017286,0.012496,0.000541,52247,0.000517,0.008288,0.006718,0.000478,both
4,FL,Altamonte Springs,42296.0,0.007937,0.072276,0.058247,0.003913,44582,0.0024,0.031582,0.027186,0.001907,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8165,WY,Sheridan,NaN,NaN,NaN,NaN,NaN,17895,0.000503,0.02062,0.015535,0.000894,right_only
8166,WY,Thermopolis,NaN,NaN,NaN,NaN,NaN,2830,0.004594,0.012014,0.007774,0.001767,right_only
8167,WY,Torrington,NaN,NaN,NaN,NaN,NaN,6709,0.001938,0.007155,0.005962,0.0,right_only
8168,WY,Wheatland,NaN,NaN,NaN,NaN,NaN,3544,0.001975,0.020316,0.012698,0.000847,right_only


In [39]:
crime_merged_left = crime_df_merged[crime_df_merged['_merge'] == 'left_only'][['State', 'City', 'population_x', 'violent_crime_x', 'prop_crime_x', 'larceny_x', 'vehicle_theft_x']]
crime_merged_left.rename(columns = {'population_x': 'population', 'violent_crime_x': 'violent_crime', 'prop_crime_x': 'prop_crime', 'larceny_x': 'larceny', 'vehicle_theft_x': 'vehicle_theft'}, inplace=True)
crime_merged_left

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
8,IA,Ankeny,46482.0,0.001573,0.029807,0.022355,0.000880
13,AL,Auburn,54785.0,0.004397,0.063937,0.048311,0.001699
31,OH,Boardman,40608.0,0.002608,0.102970,0.083141,0.003275
60,MO,Chesterfield,47560.0,0.001018,0.033379,0.029487,0.000885
73,TX,Conroe,59856.0,0.006168,0.057526,0.041896,0.003154
...,...,...,...,...,...,...,...
922,NV,Las Vegas,627244.0,0.000987,0.004693,0.002607,0.000802
925,NM,Albuquerque,561375.0,0.002439,0.013121,0.008463,0.002438
934,PA,Philadelphia,1603797.0,0.000591,0.001910,0.001432,0.000217
935,PA,Pittsburgh,305932.0,0.002145,0.010180,0.007733,0.000727


In [40]:
crime_merged_right = crime_df_merged[crime_df_merged['_merge'] == 'right_only'][['State', 'City', 'population_y', 'violent_crime_y', 'prop_crime_y', 'larceny_y', 'vehicle_theft_y']]
crime_merged_right.rename(columns = {'population_y': 'population', 'violent_crime_y': 'violent_crime', 'prop_crime_y': 'prop_crime', 'larceny_y': 'larceny', 'vehicle_theft_y': 'vehicle_theft'}, inplace=True)
crime_merged_right

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
950,AK,Bethel,6544,0.019866,0.020171,0.012836,0.004279
951,AK,Bristol Bay Borough,852,0.002347,0.023474,0.00939,0.008216
952,AK,Cordova,2150,0.0,0.003256,0.002791,0.0
953,AK,Craig,1313,0.005331,0.015232,0.009139,0.002285
954,AK,Dillingham,2405,0.020374,0.024116,0.015385,0.004158
...,...,...,...,...,...,...,...
8165,WY,Sheridan,17895,0.000503,0.02062,0.015535,0.000894
8166,WY,Thermopolis,2830,0.004594,0.012014,0.007774,0.001767
8167,WY,Torrington,6709,0.001938,0.007155,0.005962,0.0
8168,WY,Wheatland,3544,0.001975,0.020316,0.012698,0.000847


In [41]:
crime_merged_both = crime_df_merged[crime_df_merged['_merge'] == 'both'][['State', 'City', 'population_y', 'violent_crime_y', 'prop_crime_y', 'larceny_y', 'vehicle_theft_y']]
crime_merged_both.rename(columns = {'population_y': 'population', 'violent_crime_y': 'violent_crime', 'prop_crime_y': 'prop_crime', 'larceny_y': 'larceny', 'vehicle_theft_y': 'vehicle_theft'}, inplace=True)
crime_merged_both

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,PA,"Abington Township, Montgomery County",55476,0.000955,0.016836,0.015917,0.000342
1,OR,Albany,54993,0.001273,0.026676,0.022585,0.001764
2,LA,Alexandria,46630,0.015698,0.089642,0.065537,0.004975
3,CA,Aliso Viejo,52247,0.000517,0.008288,0.006718,0.000478
4,FL,Altamonte Springs,44582,0.0024,0.031582,0.027186,0.001907
...,...,...,...,...,...,...,...
945,TX,Plano,291611,0.001478,0.016831,0.013137,0.001533
946,TX,San Antonio,1559166,0.007085,0.043242,0.033011,0.00499
947,VA,Virginia Beach,449038,0.001294,0.017607,0.015137,0.001289
948,WA,Seattle,763706,0.005854,0.044956,0.030742,0.004773


In [42]:
crime_df_merged = pd.concat([crime_merged_left, crime_merged_right, crime_merged_both], ignore_index = True)
crime_df_merged

,State,City,population,violent_crime,prop_crime,larceny,vehicle_theft
0,IA,Ankeny,46482.0,0.001573,0.029807,0.022355,0.00088
1,AL,Auburn,54785.0,0.004397,0.063937,0.048311,0.001699
2,OH,Boardman,40608.0,0.002608,0.10297,0.083141,0.003275
3,MO,Chesterfield,47560.0,0.001018,0.033379,0.029487,0.000885
4,TX,Conroe,59856.0,0.006168,0.057526,0.041896,0.003154
...,...,...,...,...,...,...,...
8165,TX,Plano,291611,0.001478,0.016831,0.013137,0.001533
8166,TX,San Antonio,1559166,0.007085,0.043242,0.033011,0.00499
8167,VA,Virginia Beach,449038,0.001294,0.017607,0.015137,0.001289
8168,WA,Seattle,763706,0.005854,0.044956,0.030742,0.004773


### Merge housing and crime datasets together

In [43]:
housing_crime_df = cleaned_housing_data.merge(crime_df_merged, on = ['City', 'State'])
housing_crime_df = housing_crime_df.drop(columns = ['population_x'])
housing_crime_df

,State,City,Street,Zipcode,Bedroom,Bathroom,Area,PPSq,LotArea,MarketEstimate,...,id,house_coordinates,closest_city,city_coordinates,distance_to_maj_city_miles,population_y,violent_crime,prop_crime,larceny,vehicle_theft
0,AL,Montgomery,Brampton Ln,36117.0,3.0,2.0,1564.0,96.547315,0.2000,150500.0,...,1840008353,"(32.372746, -86.165115)",Birmingham,"(33.5279, -86.7971)",87.664428,209018.0,0.001893,0.0276,0.017277,0.00252
1,AL,Montgomery,Crestview Ave,36109.0,3.0,2.0,2196.0,76.958106,0.6100,124700.0,...,1840008353,"(32.39595, -86.269714)",Birmingham,"(33.5279, -86.7971)",83.806091,209018.0,0.001893,0.0276,0.017277,0.00252
2,AL,Montgomery,Narrow Lane Rd,36116.0,3.0,3.0,3693.0,54.156512,6.9000,188300.0,...,1840008353,"(32.273514, -86.25934)",Birmingham,"(33.5279, -86.7971)",91.920847,209018.0,0.001893,0.0276,0.017277,0.00252
3,AL,Montgomery,E Delano Ave,36105.0,3.0,2.0,1428.0,90.966387,0.2600,127500.0,...,1840008353,"(32.337, -86.29974)",Birmingham,"(33.5279, -86.7971)",87.007986,209018.0,0.001893,0.0276,0.017277,0.00252
4,AL,Montgomery,George B Edmondson Dr,36110.0,3.0,1.0,1050.0,65.714286,0.1600,66500.0,...,1840008353,"(32.40325, -86.28537)",Birmingham,"(33.5279, -86.7971)",83.007626,209018.0,0.001893,0.0276,0.017277,0.00252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,WY,Green River,Colorado Dr,82935.0,5.0,3.0,3068.0,128.748370,0.2200,381000.0,...,1840020100,"(41.494328, -109.45258)",Provo,"(40.2457, -111.6457)",143.591563,11927,0.001845,0.008133,0.007043,0.000168
9632,WY,Evansville,Henrie Roadway,82636.0,2.0,2.0,2008.0,343.625498,4.6000,689900.0,...,1840021290,"(42.86057, -106.10931)",Denver,"(39.762, -104.8758)",223.247715,2977,0.002351,0.024521,0.017467,0.002687
9633,WY,Evansville,6th St,82636.0,4.0,2.0,2022.0,157.270030,0.1828,311100.0,...,1840021290,"(42.86506, -106.27643)",Denver,"(39.762, -104.8758)",226.191583,2977,0.002351,0.024521,0.017467,0.002687
9634,WY,Evansville,6th St,82636.0,3.0,3.0,2300.0,143.434783,0.1739,329500.0,...,1840021290,"(42.86544, -106.27745)",Denver,"(39.762, -104.8758)",226.233434,2977,0.002351,0.024521,0.017467,0.002687


In [66]:
housing_crime_df.to_csv('clean_housing_data_with_crime_rates.csv', index = False)

In [67]:
housing_crime_df.columns

Index(['State', 'City', 'Street', 'Zipcode', 'Bedroom', 'Bathroom', 'Area',
       'PPSq', 'LotArea', 'MarketEstimate', 'RentEstimate', 'Latitude',
       'Longitude', 'ListedPrice', 'density', 'source', 'military',
       'incorporated', 'timezone', 'ranking', 'zips', 'id',
       'house_coordinates', 'closest_city', 'city_coordinates',
       'distance_to_maj_city_miles', 'population_y', 'violent_crime',
       'prop_crime', 'larceny', 'vehicle_theft'],
      dtype='object')